In [15]:
#importing modules
import pandas as pd
import numpy as np

In [16]:
#read csv data
user_review_data = pd.read_csv("./googleplaystore_user_reviews.csv", index_col=None)

#setting seed for reproducibility
np.random.seed(0)

In [17]:
#creating a data frame for data
df_user_review = pd.DataFrame(user_review_data)
df_user_review = df_user_review.dropna()
df_user_review.dtypes

App                        object
Translated_Review          object
Sentiment                  object
Sentiment_Polarity        float64
Sentiment_Subjectivity    float64
dtype: object

In [18]:
#setting the index
df_user_review['review_app_id'] = range(1, len(df_user_review) + 1)
df_user_review = df_user_review.set_index('review_app_id')

In [19]:
#creating a new dataframe for reviews table
column_names = ['app_name','user_reviews']
reviews = df_user_review[['App','Translated_Review']]
df_reviews = pd.DataFrame(reviews)
df_reviews.columns = column_names
df_reviews.dropna()
df_reviews.tail()

,app_name,user_reviews
review_app_id,,
37423,Housing-Real Estate & Property,Most ads older many agents ..not much owner po...
37424,Housing-Real Estate & Property,"If photos posted portal load, fit purpose. I'm..."
37425,Housing-Real Estate & Property,"Dumb app, I wanted post property rent give opt..."
37426,Housing-Real Estate & Property,I property business got link SMS happy perform...
37427,Housing-Real Estate & Property,"Useless app, I searched flats kondapur, Hydera..."


In [20]:
#creating a new dataframe for sentiments table
column_names = ['app_name','sentiment', 'polarity', 'subjectivity']
sentiments = df_user_review[['App','Sentiment','Sentiment_Polarity','Sentiment_Subjectivity']]
df_sentiment = pd.DataFrame(sentiments)
df_sentiment.columns = column_names
df_sentiment = df_sentiment.dropna()

df_sentiment.tail()

,app_name,sentiment,polarity,subjectivity
review_app_id,,,,
37423,Housing-Real Estate & Property,Positive,0.173333,0.486667
37424,Housing-Real Estate & Property,Positive,0.225000,0.447222
37425,Housing-Real Estate & Property,Negative,-0.287500,0.250000
37426,Housing-Real Estate & Property,Positive,0.800000,1.000000
37427,Housing-Real Estate & Property,Negative,-0.316667,0.400000


In [21]:
#python database connectivity
import psycopg2

#establishing the connection
conn = psycopg2.connect(
   database="shreyachoure", user='shreyachoure', password='root', host='127.0.0.1', port= '5432'
)
#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Executing an MYSQL function using the execute() method
cursor.execute("select version()")

# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)

#Closing the connection
conn.close()

Connection established to:  ('PostgreSQL 14.5 on aarch64-apple-darwin20.6.0, compiled by Apple clang version 12.0.5 (clang-1205.0.22.9), 64-bit',)


In [22]:
#Inserting data from python to psql table 'Applications'
from sqlalchemy import create_engine

#creating engine
engine = create_engine('postgresql://shreyachoure@127.0.0.1:5432/shreyachoure')

from sqlalchemy.orm import Session
with Session(engine) as session:
    engine.execute("DROP table IF EXISTS reviews")
    engine.execute("DROP table IF EXISTS sentiments")
   

    df_reviews.to_sql('reviews', con=engine)
    df_sentiment.to_sql('sentiments', con=engine)

    session.commit()